In [23]:
import numpy as np
import matplotlib.pyplot as plt

import sys
print(sys.executable)


def change_point_model(trial_by_global_param = None, trial_by_logic_param = 'NaN'):
    observations = ['Dev at 1', 'Dev at 2','Dev at 3','Dev at 4','Dev at 5','Dev at 6', 'No deviant']

    # Step 1: Initialize prior belief (biased towards position 4)
    def initialize_priors():
        deviant_given_sequence = np.array([1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7])  # Example prior belief
        global_prior = np.array([1/3, 1/3, 1/3])
        return deviant_given_sequence, global_prior

    # Step 2: Initialize likelihood matrices
    def initialize_likelihoods():
        trial_by_global = trial_by_global_param

        if(trial_by_logic_param is None):
            trial_logic = np.array([
                #HM
                [0, 0, 0, 0.33, 0.33, 0.33, 0],
                #FC
                [0, 0, 0, 0, 0, 0, 1]
            ]).T
        else:
            trial_logic = trial_by_logic_param
            
        return trial_by_global, trial_logic

    # Step 3: Initialize helper functions
    def sequence_given_state(sequence, trial_by_global, trial_logic):
        deviant_given_state = np.ones(3)
        for i in range(len(sequence)):
            deviant_given_trial = np.array([trial_logic[sequence[i]][0], trial_logic[sequence[i]][1]])
            deviant_given_state *= deviant_given_trial @ trial_by_global

        return deviant_given_state

    def state_given_sequence(sequence, prior, trial_by_global, trial_logic):
        SEQUENCE_GIVEN_STATE = sequence_given_state(sequence, trial_by_global, trial_logic)
        state_prob = prior.copy()
        state_prob = SEQUENCE_GIVEN_STATE * state_prob
        norm_state_prob = state_prob / state_prob.sum()
        return norm_state_prob

    def future_given_state(trial_by_global, trial_logic):

        return trial_logic @ trial_by_global

    def future_given_sequence(sequence, prior, future_given_state, trial_by_global, trial_logic):
        FUTURE_GIVEN_STATE = future_given_state
        STATE_GIVEN_SEQUENCE = state_given_sequence(sequence, prior, trial_by_global, trial_logic)    
        
        return FUTURE_GIVEN_STATE @ STATE_GIVEN_SEQUENCE.T



    # Step 4: Gradually update the prior based on the observed deviant location with recency-based decay
    def update_priors_simplified(deviant_given_sequence, global_prior, future_given_state, trial_by_global, trial_logic, deviant_position, obs_history):    
        obs_history.append(deviant_position)

        size = len(obs_history)
        index = (size // 250) * 250 if (size // 250) != 0 else (size // 250) * 250 + 1

        clipped_obs_history = obs_history[index - 1: size].copy()

        deviant_given_sequence = future_given_sequence(clipped_obs_history, global_prior, future_given_state, trial_by_global, trial_logic)

        return deviant_given_sequence, obs_history

    # Step 5: Run trials and update beliefs
    def run_standard_trials(deviant_given_sequence = None, global_prior = None, belief_over_time = None, obs_history = None, n_trials = 1000):
        if (deviant_given_sequence is None or belief_over_time is None or obs_history is None):
            belief_over_time = []  # Store belief at each trial
            obs_history = []  # Track history of observed deviants
        
            # Initialize prior belief (even distribution across all positions initially)
            deviant_given_sequence, global_prior = initialize_priors()
        else:
            belief_over_time = belief_over_time  # Store belief at each trial
            obs_history = obs_history  # Track history of observed deviants
            
            # Initialize prior belief (even distribution across all positions initially)
            deviant_given_sequence = deviant_given_sequence
            global_prior = global_prior
        
        trial_by_global, trial_logic = initialize_likelihoods()
        
        FUTURE_GIVEN_STATE = future_given_state(trial_by_global, trial_logic)

        # Perform the trials
        for trial in range(n_trials):        
            # Randomly choose where the deviant is located (uniform distribution over 6 positions)
            deviant_position = np.random.randint(3, 6)
            
            # After each trial, update the prior based on the deviant position observed in the current trial (gradual update)
            deviant_given_sequence, obs_history = update_priors_simplified(deviant_given_sequence, global_prior, FUTURE_GIVEN_STATE, trial_by_global, trial_logic, deviant_position, obs_history)
            
            # Record the belief after this trial
            belief_over_time.append(deviant_given_sequence.copy())
        
        return deviant_given_sequence, belief_over_time, obs_history
    
    # Step 5: Run trials and update beliefs
    def run_probe_trials(deviant_given_sequence = None, global_prior = None, belief_over_time = None, obs_history = None, n_trials = 1000):
        if (deviant_given_sequence is None or belief_over_time is None or obs_history is None):
            belief_over_time = []  # Store belief at each trial
            obs_history = []  # Track history of observed deviants
        
            # Initialize prior belief (even distribution across all positions initially)
            deviant_given_sequence, global_prior = initialize_priors()
        else:
            belief_over_time = belief_over_time  # Store belief at each trial
            obs_history = obs_history  # Track history of observed deviants
            
            # Initialize prior belief (even distribution across all positions initially)
            deviant_given_sequence = deviant_given_sequence
            global_prior = global_prior
        
        trial_by_global, trial_logic = initialize_likelihoods()
        
        FUTURE_GIVEN_STATE = future_given_state(trial_by_global, trial_logic)

        # Perform the trials
        for trial in range(n_trials):        
            # Randomly choose where the deviant is located (uniform distribution over 6 positions)
            positions = [3, 4, 5, 6]
            probabilities = [0.95/3, 0.95/3, 0.95/3, 0.05]  # 60% for positions 3, 4, 5; 40% for position 7
            deviant_position = np.random.choice(positions, size=1, p=probabilities)[0]
            
            # After each trial, update the prior based on the deviant position observed in the current trial (gradual update)
            deviant_given_sequence, obs_history = update_priors_simplified(deviant_given_sequence, global_prior, FUTURE_GIVEN_STATE, trial_by_global, trial_logic, deviant_position, obs_history)
            
            # Record the belief after this trial
            belief_over_time.append(deviant_given_sequence.copy())
        
        return deviant_given_sequence, belief_over_time, obs_history

    # Step 5: Run trials and update beliefs
    def run_catch_trials(deviant_given_sequence = None, global_prior = None, belief_over_time = None, obs_history = None, n_trials = 1000):
        if (deviant_given_sequence is None or belief_over_time is None or obs_history is None):
            belief_over_time = []  # Store belief at each trial
            obs_history = []  # Track history of observed deviants
        
            # Initialize prior belief (even distribution across all positions initially)
            deviant_given_sequence, global_prior = initialize_priors()
        else:
            belief_over_time = belief_over_time  # Store belief at each trial
            obs_history = obs_history  # Track history of observed deviants
            
            # Initialize prior belief (even distribution across all positions initially)
            deviant_given_sequence = deviant_given_sequence
            global_prior = global_prior
        
        trial_by_global, trial_logic = initialize_likelihoods()
        
        FUTURE_GIVEN_STATE = future_given_state(trial_by_global, trial_logic)

        # Perform the trials
        for trial in range(n_trials):        
            positions = [3, 4, 5, 6]
            probabilities = [0.6/3, 0.6/3, 0.6/3, 0.4]  # 60% for positions 3, 4, 5; 40% for position 7
            deviant_position = np.random.choice(positions, size=1, p=probabilities)[0]
            
            # After each trial, update the prior based on the deviant position observed in the current trial (gradual update)
            deviant_given_sequence, obs_history = update_priors_simplified(deviant_given_sequence, global_prior, FUTURE_GIVEN_STATE, trial_by_global, trial_logic, deviant_position, obs_history)
            
            # Record the belief after this trial
            belief_over_time.append(deviant_given_sequence.copy())
        
        return deviant_given_sequence, belief_over_time, obs_history






    standard_prior_belief, standard_belief_over_time, standard_obs_history = run_standard_trials(None, None, None, None, n_trials = 1500)

    global_prior = np.array([1/3, 1/3, 1/3])

    for i in range(5):
        standard_prior_belief, standard_belief_over_time, standard_obs_history = run_standard_trials(standard_prior_belief.copy(), global_prior, standard_belief_over_time.copy(), standard_obs_history.copy(), n_trials = 250)
        probe_prior_belief, probe_belief_over_time, probe_obs_history = run_probe_trials(standard_prior_belief.copy(), global_prior, standard_belief_over_time.copy(), standard_obs_history.copy(), n_trials = 250)
        standard_prior_belief, standard_belief_over_time, standard_obs_history = run_standard_trials(probe_prior_belief.copy(), global_prior, probe_belief_over_time.copy(), probe_obs_history.copy(), n_trials = 500)
        probe_prior_belief, probe_belief_over_time, probe_obs_history = run_probe_trials(standard_prior_belief.copy(), global_prior, standard_belief_over_time.copy(), standard_obs_history.copy(), n_trials = 250)
        standard_prior_belief, standard_belief_over_time, standard_obs_history = run_standard_trials(probe_prior_belief.copy(), global_prior, probe_belief_over_time.copy(), probe_obs_history.copy(), n_trials = 250)

    catch_prior_belief, catch_belief_over_time, catch_deviant_positions = run_catch_trials(probe_prior_belief.copy(), global_prior, probe_belief_over_time.copy(), probe_obs_history.copy(), n_trials = 1500)

    return catch_belief_over_time

c:\Users\Fatbu\anaconda3\python.exe


# Un/Bounded HM/FC & A/Symmetric HM/FC.  Not included stochastic block & no HM free variable

### Bounds on HM/FC & Symmetric HM/FC, No Stochastic Block

In [ ]:
from bayes_opt import BayesianOptimization
import math

def standard_obs(data, trial_count):
    for i in range(trial_count):
        deviant_position = np.random.randint(3, 6)
        data = np.append(data, deviant_position)
    return data

def probe_obs(data, trial_count):
    for i in range(trial_count):
        positions = [3, 4, 5, 6]
        probabilities = [0.95/3, 0.95/3, 0.95/3, 0.05]  
        deviant_position = np.random.choice(positions, size=1, p=probabilities)[0]
        data = np.append(data, deviant_position)
    return data

def catch_obs(data, trial_count):
    for i in range(trial_count):
        positions = [3, 4, 5, 6]
        probabilities = [0.6/3, 0.6/3, 0.6/3, 0.4] 
        deviant_position = np.random.choice(positions, size=1, p=probabilities)[0]
        data = np.append(data, deviant_position)
    return data

data = np.array([])

data = standard_obs(data, 1500)
for i in range(5):
    data = standard_obs(data, 250)
    data = probe_obs(data, 250)
    data = standard_obs(data, 500)
    data = probe_obs(data, 250)
    data = standard_obs(data, 250)
data = catch_obs(data, 1500)

print(data)


def neg_log_likelihood(hm_standard, hm_probe, hm_catch):
    trial_by_global = np.array([
        [hm_standard, 1 - hm_standard],
        [hm_probe, 1 - hm_probe],
        [hm_catch, 1 - hm_catch]
    ]).T

    priors = change_point_model(trial_by_global)
    
    nll = 0
    for i in range(len(priors)): ## priors and data must be same len
        nll += -math.log(priors[i][int(data[i])])

    return -nll #have to negate since bayesian optimizer is a maximizer not a minimizer

# Define bounds for each parameter (between 0.01 and 0.99 to avoid degenerate values)
pbounds = {
    'hm_standard': (0.5, 0.99),
    'hm_probe': (0.5, 0.98),
    'hm_catch': (0.3, 0.7)
}

# Initialize the optimizer
optimizer = BayesianOptimization(
    f=neg_log_likelihood,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Run optimization
optimizer.maximize(
    init_points=10,  # random samples before GP fitting
    n_iter=30        # number of iterations of Bayesian optimization
)    

[4. 4. 3. ... 6. 6. 5.]
|   iter    |  target   | hm_sta... | hm_probe  | hm_catch  |
-------------------------------------------------------------
| 1         | -12407.17 | 0.6835246 | 0.9563428 | 0.5927975 |
| 2         | -13870.61 | 0.7933426 | 0.5748889 | 0.3623978 |
| 3         | -12753.93 | 0.5284609 | 0.9157645 | 0.5404460 |
| 4         | -13357.29 | 0.8469555 | 0.5098805 | 0.6879639 |
| 5         | -12783.70 | 0.9078968 | 0.6019227 | 0.3727299 |
| 6         | -15574.36 | 0.5898682 | 0.6460362 | 0.5099025 |
| 7         | -14752.68 | 0.7116530 | 0.6397899 | 0.5447411 |
| 8         | -15653.62 | 0.5683519 | 0.6402294 | 0.4465447 |
| 9         | -13065.46 | 0.7234742 | 0.8768844 | 0.3798695 |
| 10        | -13973.67 | 0.7519748 | 0.7843589 | 0.3185801 |
| 11        | -12874.46 | 0.8961147 | 0.6654977 | 0.6250920 |
| 12        | -12314.30 | 0.99      | 0.98      | 0.5026171 |
| 13        | -12230.07 | 0.99      | 0.98      | 0.7       |
| 14        | -12374.25 | 0.5       | 0.98    

### Bounds on HM/FC & Asymmetric HM/FC, No Stochastic Block

In [13]:
def neg_log_likelihood(hm_standard, hm_probe, hm_catch, fc_standard, fc_probe, fc_catch):
    trial_by_global = np.array([
        [hm_standard, fc_standard],
        [hm_probe, fc_probe],
        [hm_catch, fc_catch]
    ]).T

    priors = change_point_model(trial_by_global)
    
    nll = 0
    for i in range(len(priors)): ## priors and data must be same len
        nll += -math.log(priors[i][int(data[i])])

    return -nll #have to negate since bayesian optimizer is a maximizer not a minimizer

# Define bounds for each parameter (between 0.01 and 0.99 to avoid degenerate values)
pbounds = {
    'hm_standard': (0.5, 0.99),
    'hm_probe': (0.5, 0.98),
    'hm_catch': (0.3, 0.7),
    'fc_standard': (0.01, 0.5),
    'fc_probe': (0.01, 0.5),
    'fc_catch': (0.01, 0.5)
}

# Initialize the optimizer
optimizer = BayesianOptimization(
    f=neg_log_likelihood,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Run optimization
optimizer.maximize(
    init_points=10,  # random samples before GP fitting
    n_iter=30        # number of iterations of Bayesian optimization
)

|   iter    |  target   | hm_sta... | hm_probe  | hm_catch  | fc_sta... | fc_probe  | fc_catch  |
-------------------------------------------------------------------------------------------------
| 1         | -12357.75 | 0.6835246 | 0.9563428 | 0.5927975 | 0.3033426 | 0.0864491 | 0.0864373 |
| 2         | -12895.02 | 0.5284609 | 0.9157645 | 0.5404460 | 0.3569555 | 0.0200864 | 0.4852558 |
| 3         | -13113.02 | 0.9078968 | 0.6019227 | 0.3727299 | 0.0998682 | 0.1590786 | 0.2671306 |
| 4         | -15346.19 | 0.7116530 | 0.6397899 | 0.5447411 | 0.0783519 | 0.1531508 | 0.1895173 |
| 5         | -12720.50 | 0.7234742 | 0.8768844 | 0.3798695 | 0.2619748 | 0.3002831 | 0.0327607 |
| 6         | -13345.82 | 0.7976969 | 0.5818515 | 0.3260206 | 0.4749539 | 0.4831596 | 0.4061147 |
| 7         | -15802.38 | 0.6492607 | 0.5468826 | 0.5736932 | 0.2256747 | 0.0697987 | 0.2526366 |
| 8         | -12468.55 | 0.5168503 | 0.9364737 | 0.4035119 | 0.3346359 | 0.1627384 | 0.2648333 |
| 9         | -13844

### Unbounded HM/FC & Symmetric HM/FC, No Stochastic Block

In [18]:
def neg_log_likelihood(hm_standard, hm_probe, hm_catch):
    trial_by_global = np.array([
        [hm_standard, 1- hm_standard],
        [hm_probe, 1 - hm_probe],
        [hm_catch, 1 - hm_catch]
    ]).T

    priors = change_point_model(trial_by_global)
    
    nll = 0
    for i in range(len(priors)): ## priors and data must be same len
        nll += -math.log(priors[i][int(data[i])])

    return -nll #have to negate since bayesian optimizer is a maximizer not a minimizer

# Define bounds for each parameter (between 0.01 and 0.99 to avoid degenerate values)
pbounds = {
    'hm_standard': (0.01, 0.99),
    'hm_probe': (0.01, 0.99),
    'hm_catch': (0.01, 0.99)
}

# Initialize the optimizer
optimizer = BayesianOptimization(
    f=neg_log_likelihood,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Run optimization
optimizer.maximize(
    init_points=10,  # random samples before GP fitting
    n_iter=30        # number of iterations of Bayesian optimization
)

|   iter    |  target   | hm_sta... | hm_probe  | hm_catch  |
-------------------------------------------------------------
| 1         | -12563.41 | 0.3770493 | 0.9417000 | 0.7273540 |
| 2         | -16231.81 | 0.5966853 | 0.1628982 | 0.1628746 |


KeyboardInterrupt: 

### Unbounded HM/FC & Asymmetric HM/FC, No Stochastic Block

In [20]:
def neg_log_likelihood(hm_standard, hm_probe, hm_catch, fc_standard, fc_probe, fc_catch):
    trial_by_global = np.array([
        [hm_standard, fc_standard],
        [hm_probe, fc_probe],
        [hm_catch, fc_catch]
    ]).T

    priors = change_point_model(trial_by_global)
    
    nll = 0
    for i in range(len(priors)): ## priors and data must be same len
        nll += -math.log(priors[i][int(data[i])])

    return -nll #have to negate since bayesian optimizer is a maximizer not a minimizer

# Define bounds for each parameter (between 0.01 and 0.99 to avoid degenerate values)
pbounds = {
    'hm_standard': (0.01, 0.99),
    'hm_probe': (0.01, 0.99),
    'hm_catch': (0.01, 0.99),
    'fc_standard': (0.01, 0.99),
    'fc_probe': (0.01, 0.99),
    'fc_catch': (0.01, 0.99),
}

# Initialize the optimizer
optimizer = BayesianOptimization(
    f=neg_log_likelihood,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Run optimization
optimizer.maximize(
    init_points=10,  # random samples before GP fitting
    n_iter=30        # number of iterations of Bayesian optimization
)

|   iter    |  target   | hm_sta... | hm_probe  | hm_catch  | fc_sta... | fc_probe  | fc_catch  |
-------------------------------------------------------------------------------------------------
| 1         | -12419.67 | 0.3770493 | 0.9417000 | 0.7273540 | 0.5966853 | 0.1628982 | 0.1628746 |
| 2         | -12942.17 | 0.0669219 | 0.8588526 | 0.5990927 | 0.7039111 | 0.0301728 | 0.9605116 |
| 3         | -13576.38 | 0.8257937 | 0.2180923 | 0.1881884 | 0.1897364 | 0.3081573 | 0.5242613 |
| 4         | -16091.52 | 0.4333061 | 0.2954045 | 0.6096158 | 0.1467039 | 0.2963017 | 0.3690346 |
| 5         | -13425.39 | 0.4569485 | 0.7794724 | 0.2056803 | 0.5139497 | 0.5905662 | 0.0555214 |
| 6         | -15561.00 | 0.6053939 | 0.1771136 | 0.0737505 | 0.9399078 | 0.9563193 | 0.8022294 |
| 7         | -14835.61 | 0.3085214 | 0.1057186 | 0.6805483 | 0.4413494 | 0.1295974 | 0.4952733 |
| 8         | -12412.03 | 0.0437007 | 0.9011339 | 0.2636043 | 0.6592718 | 0.3154768 | 0.5196666 |
| 9         | -11163

# HM Free Variable

In [1]:
def neg_log_likelihood(hm_standard, hm_probe, hm_catch, dev4, dev5):
    trial_by_global = np.array([
        [hm_standard, 1 - hm_standard],
        [hm_probe, 1 - hm_probe],
        [hm_catch, 1 - hm_catch]
    ]).T
    trial_by_logic = np.array([
        [0, 0, 0, dev4, dev5, 1 - dev4 - dev5, 0],
        [0, 0, 0, 0, 0, 0, 1]
    ]).T

    priors = change_point_model(trial_by_global, trial_by_logic)
    
    nll = 0
    for i in range(len(priors)): ## priors and data must be same len
        nll += -math.log(priors[i][int(data[i])])

    return -nll #have to negate since bayesian optimizer is a maximizer not a minimizer

# Define bounds for each parameter (between 0.01 and 0.99 to avoid degenerate values)
pbounds = {
    'hm_standard': (0.5, 0.99),
    'hm_probe': (0.5, 0.98),
    'hm_catch': (0.3, 0.7),
    'dev4': (0.01, 0.49),
    'dev5': (0.01, 0.49)
}

# Initialize the optimizer
optimizer = BayesianOptimization(
    f=neg_log_likelihood,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Run optimization
optimizer.maximize(
    init_points=10,  # random samples before GP fitting
    n_iter=30        # number of iterations of Bayesian optimization
)    

NameError: name 'BayesianOptimization' is not defined